<a href="https://colab.research.google.com/github/bcalik94/Multimodal-Speech-Summarisation-/blob/main/Preprocess_4_Extractive_Summarsation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q transformers datasets rouge-score
#install spacy large language model. (Colab needs to be restarted/)
!python -m spacy download en_core_web_lg

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 412, in run
    env = get_environment(lib_locations)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/metadata/__init__.py", line 29, in get_environment
    return Environment.from_paths(paths)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/metadata/pkg_resources.py", line 84, in from_paths
    return cls(pkg_resources.WorkingSet(paths))
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 567, in __init__
    self.add_entry(entry)
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 623, 

In [ ]:
pip install clean-text

     |████████████████████████████████| 53 kB 1.3 MB/s 
     |████████████████████████████████| 175 kB 9.8 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=b039e2f55247dee9da25665951fa3138a22c94a9ea704513852a6cc415c0ffc1
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Not connected to a GPU
Your runtime has 37.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import tensorflow_datasets as tfds 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from tqdm.notebook import tqdm
  
import tensorflow_hub as hub
from tensorflow import keras 
import tensorflow as tf
from IPython.display import Image 
import matplotlib.pyplot as plt

import json
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import os

from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from datetime import datetime as dt
from spacy.lang.en import English 
from __future__ import unicode_literals, print_function
import nltk
nltk.download('punkt')
import math


nlp = spacy.load('en_core_web_lg')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

'''
Combine podcast transcript, metadata and filter the datasets for further process
- Find the links for each podcast episode's transcript and time-stamped acoustic info

'''
from tqdm import tqdm
from pathlib import Path
import json
import nltk
nltk.download('punkt')
#import clean function
from cleantext import clean

opensmile_directory_path = '/content/drive/MyDrive/OpenSmile_List'
podcast_directory_path = '/content/drive/MyDrive/Podcast_List'


podcast_rootdir = Path(podcast_directory_path)
podcastfile_list = [f for f in podcast_rootdir.glob('**/*') if f.is_file()]
    
opensmile_rootdir = Path(opensmile_directory_path)
opensmilefile_list = [f for f in opensmile_rootdir.glob('**/*') if f.is_file()] 

df=[]

for x, y in tqdm([(x,y) for x in opensmilefile_list for y in podcastfile_list]):
      if Path(y).stem == Path(x).stem:
        #print("spotify:episode:" + Path(y).stem,"spotify:episode:" + Path(x).stem)
        transcript = ""
        count = 0
        episode_id = "spotify:episode:" + Path(y).stem
        with open(str(y)) as json_file:
            data = json.load(json_file)
            for item in data['results']:
                try:
                    transcript += ' ' + item['alternatives'][0]['transcript']
                    count+=1
                except:
                    pass 
                     #not all item with 'alternatives' key contain a 'transcript' key
        sentences = nltk.sent_tokenize(transcript) 
        df.append({'episode_uri': episode_id,
                   'Transcript': transcript, 
                   'opensmile_path': str(x), 
                   'podcast_path': str(y),
                   'sentence_count': len(sentences)})
                   #'First 30 sentences': ' '.join(sentences[:30]),
                   #'number_of_sentences': len(sentences)                     
df = pd.DataFrame(df)

def clean_data(dataframe, text):
#replace URL of a text
    dataframe[text] = dataframe[text].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    return dataframe 


#import the metadata
podcasts_rhs = pd.read_csv('/content/drive/MyDrive/metadata.tsv', sep='\t').dropna()
'''
Check this part agaian fo filtering
'''
#Prepre-process    
metadata_podcast = df.merge(podcasts_rhs, how='inner', on='episode_uri').dropna()
metadata_podcast.drop(metadata_podcast.columns[5], axis=1, inplace=True)

metadata_podcast = clean_data(metadata_podcast, 'episode_description')
metadata_podcast = clean_data(metadata_podcast, 'Transcript')
metadata_podcast = metadata_podcast[metadata_podcast["episode_description"].str.contains("http")==False]
metadata_podcast = metadata_podcast[metadata_podcast["language"].str.contains("en")==True]
metadata_podcast = metadata_podcast.drop_duplicates(subset = ["episode_description"])
metadata_podcast = metadata_podcast[metadata_podcast["episode_description"].str.len() > 150]
#metadata_podcast = metadata_podcast[metadata_podcast["Transcript"].str.len() > 400]
metadata_podcast = metadata_podcast[metadata_podcast["sentence_count"] > 60 ]
metadata_podcast = metadata_podcast[metadata_podcast["sentence_count"] < 750 ]

metadata_podcast['show_name'] =metadata_podcast.apply(lambda row : clean(row['show_name'], no_emoji=True), axis = 1)
metadata_podcast['episode_description'] =metadata_podcast.apply(lambda row : clean(row['episode_description'], no_emoji=True), axis = 1)

podcast_data = metadata_podcast[[ 'show_name','episode_uri', 'Transcript','episode_description', 'opensmile_path', 'podcast_path']].dropna().reset_index(drop = True)
#ADD podcast name 
podcast_data.episode_description = podcast_data.episode_description.apply(lambda x: x.encode().decode('utf-8'))
podcast_data.Transcript = podcast_data.Transcript.apply(lambda x: x.encode().decode('utf-8')) 
podcast_data.episode_uri = podcast_data.episode_uri.apply(lambda x: x.encode().decode('utf-8')) 
#podcast_data = podcast_data.sample(n=1000)

print(" ------------------------------------------------------------------------------")
print(" ------------------------------------------------------------------------------")
print("Total number of episodes before prepreprocessing: "+ str(len(df)))
print("Total number of episodes after prepreprocessing: "+ str(len(podcast_data)))
print("Total number of podcasts after prepreprocessing: " + str(len(podcast_data.groupby('show_name').count())) )
print(" ------------------------------------------------------------------------------")
print(" ------------------------------------------------------------------------------")
print(" ------------ Episode Info ----------------------------------------------------")
podcast_data.groupby('show_name').count().head()




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


100%|██████████| 102912866/102912866 [1:03:44<00:00, 26907.26it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: FutureWarning: The default value of regex will change from True to False in a future version.


 ------------------------------------------------------------------------------
 ------------------------------------------------------------------------------
Total number of episodes before prepreprocessing: 7321
Total number of episodes after prepreprocessing: 4197
Total number of podcasts after prepreprocessing: 933
 ------------------------------------------------------------------------------
 ------------------------------------------------------------------------------
 ------------ Episode Info ----------------------------------------------------


,episode_uri,Transcript,episode_description,opensmile_path,podcast_path
show_name,,,,,
"""are you going to be ok?"" with chris baker",1,1,1,1,1
#5things,1,1,1,1,1
#askreezy with reezy resells,2,2,2,2,2
#dobetter pod,3,3,3,3,3
#lakers podcast,1,1,1,1,1


In [ ]:
podcast_data

,show_name,episode_uri,Transcript,episode_description,opensmile_path,podcast_path
0,2 in a boat,spotify:episode:7AZDgHltNTeGFyqUB6vDJN,Welcome to the to in a boat podcast starring ...,"on a wet windy june day, suzanne and andrew di...",/content/drive/MyDrive/OpenSmile_List/E/show_6...,/content/drive/MyDrive/Podcast_List/E/show_6E2...
1,vegan 101,spotify:episode:4ReExO172VOTxorndX8Hfz,"Hello, my name is Kendra and I'm the host of ...",vegan 101 revolves all around making veganism ...,/content/drive/MyDrive/OpenSmile_List/E/show_6...,/content/drive/MyDrive/Podcast_List/E/show_6eb...
2,the very good podcast,spotify:episode:3ThCh3ko96O186PYt5qmHi,I love the idea of Family Worship. Where can ...,presenting 'the very good podcast' from atlant...,/content/drive/MyDrive/OpenSmile_List/E/show_6...,/content/drive/MyDrive/Podcast_List/E/show_6eA...
3,mental dumbells,spotify:episode:72Wl0Iv7bzRNU7BBpFiBh6,This episode of mental dumbbells. We talk abo...,01.30 - disclaimer for this episode 02.00 - ho...,/content/drive/MyDrive/OpenSmile_List/E/show_6...,/content/drive/MyDrive/Podcast_List/E/show_6E2...
4,mental dumbells,spotify:episode:7N15RDhAn1aHON8bbnSs8l,So you create McDonald's and it's been 20 min...,in this episode we talk all about starting a c...,/content/drive/MyDrive/OpenSmile_List/E/show_6...,/content/drive/MyDrive/Podcast_List/E/show_6E2...
...,...,...,...,...,...,...
4192,bayts,spotify:episode:5T0VzdB3Pj51MBKkcgcCbt,"Hi, welcome back to page. Hi, welcome back to...",today we talk about the business side of youtu...,/content/drive/MyDrive/OpenSmile_List/D/show_6...,/content/drive/MyDrive/Podcast_List/D/show_6dz...
4193,bayts,spotify:episode:2J77TfrKNMVahO5bYcVems,"Hey guys, welcome back to Bates the podcast w...",how people think it's okay to make false accus...,/content/drive/MyDrive/OpenSmile_List/D/show_6...,/content/drive/MyDrive/Podcast_List/D/show_6dz...
4194,bayts,spotify:episode:6tduAbqoS1fig0ddUrxp6w,Hello everybody and welcome back to Bates the...,we talk about what its like being parents on y...,/content/drive/MyDrive/OpenSmile_List/D/show_6...,/content/drive/MyDrive/Podcast_List/D/show_6dz...
4195,bayts,spotify:episode:0NszSDdIhnCjL9stnNSNsr,Can can you say something in your mic? Your ...,we've been gone a bit and here's some reasons ...,/content/drive/MyDrive/OpenSmile_List/D/show_6...,/content/drive/MyDrive/Podcast_List/D/show_6dz...


In [ ]:
#Randomly select podcast episodes 

podcast_data = podcast_data.sample(1250).reset_index(drop=True)


In [ ]:
'''
- Synchronize/Tag the acoustic and linguistics info of the podcast episodes based on indicated time-window

'''
#Compute avarage acoustic features embeddings in sentence-level with senter 
from statistics import mode
from collections import Counter

def embed_senter(dataframe):
    idx = []
    speakerIDx = []
    speaker4sent = []
    sentences = []
    words4sent =[]
    min_acoustics = []
    mean_acoustics = []
    max_acoustics = []
    for index,row in dataframe.iterrows():
        if row['words'].endswith(".") or row['words'].endswith("?") or row['words'].endswith("!"):
            idx.append(index)
            words4sent.append(row['words'])
            joined = " ".join(words4sent)
            sentences.append({'sentence':joined})

            #Speaker Tagging 
            
            speakerIDx.append(row['speakerTag'])
            speaker4sent.append({'speakerTag' : Counter(speakerIDx).most_common(1)[0][0]})
            #print(joined)    #print the sentences
 
            mean = dataframe.iloc[ idx, 5: ].mean(axis=0)
            min = dataframe.iloc[ idx, 5: ].min(axis=0)
            max = dataframe.iloc[ idx, 5: ].max(axis=0)
            mean_acoustics.append(mean.to_frame().T.add_prefix('MEAN:'))
            min_acoustics.append(min.to_frame().T.add_prefix('MIN:'))
            max_acoustics.append(max.to_frame().T.add_prefix('MAX:'))
            #print(mean)
            speakerIDx = []
            idx =[]
            words4sent =[]
        else: 
         #print(row['words'])
             words4sent.append(row['words'])
             speakerIDx.append(row['speakerTag'])
             idx.append(index)

    sentences = pd.DataFrame(sentences)
    speaker4sent = pd.DataFrame(speaker4sent)
    min_merged = pd.concat(min_acoustics).reset_index(drop=True)
    max_merged = pd.concat(max_acoustics).reset_index(drop=True)
    mean_merged = pd.concat(mean_acoustics).reset_index(drop=True)
    result = sentences.join(speaker4sent).join(mean_merged).join(min_merged).join(max_merged)

    return result


def retrieve_timestamped_transcript(path_transcript, path_opensmile):
    
    
        """Load the full transcript with timestamps."""
        with open(path_transcript, "r") as file:
            transcript = json.load(file)
    
        starts, ends, words, speakers = [], [], [], []
        for word in transcript["results"][-1]["alternatives"][0]["words"]:
            starts.append(float(word["startTime"].replace("s", "")))
            ends.append(float(word["endTime"].replace("s", "")))
            words.append(word["word"])
            speakers.append(word["speakerTag"])
    
        starts = np.array(starts, dtype=np.float32)
        ends = np.array(ends, dtype=np.float32)
        words = np.array(words)
        speakers = np.array(speakers, dtype=np.int32)
        
        timestamp = pd.DataFrame(zip(starts,ends,words,speakers), columns=["starts", "ends","words","speakerTag"])
        timestamp['Centroid_time'] = (timestamp['ends'] + timestamp['starts'])/2
        

        #Read hdf file with absolute path 
        embeddings =pd.read_hdf(path_opensmile,mode='r', errors = 'ignore')
        #Reset index and bring the time data to another column where the time interval starts and ends
        embeddings = embeddings.reset_index()
        embeddings.insert(1,'ends',embeddings['time (s)'].shift(-1))
        embeddings = pd.DataFrame(embeddings.rename(columns={'time (s)': 'starts'}))
        embeddings.drop(embeddings.tail(1).index,inplace=True) # drop the last row
        
        #Synchronization - alignment word level acoustics features
        transcript_info, acoustic_info = [], []
        i = 0
        j = 0
        while timestamp['ends'][len(timestamp)-1] > embeddings['ends'][i]:
            try:
                        if timestamp['Centroid_time'][j] >= embeddings['starts'][i] and timestamp['Centroid_time'][j] <= embeddings['ends'][i] :
                                transcript_info.append(timestamp.iloc[j,:].to_frame().T)
                                #print(timestamp['words'][j])
                                acoustic_info.append(embeddings.iloc[i-1,:].to_frame().T)
                                j=j+1
                        elif i == len(embeddings)-1 :
                            break
                        else: i=i+1
                        continue
            except KeyError: break 

        
        transcript_info = pd.concat(transcript_info).reset_index(drop=True)
        
        acoustic_info = pd.concat(acoustic_info).reset_index(drop=True).drop(labels=['starts', 'ends'], axis=1)

        merged_info = transcript_info.join(acoustic_info) 
        
        embedded_info = embed_senter(merged_info)
        
        return embedded_info

In [ ]:
import pickle

with open('podcast_data.txt', 'wb') as f1:
   pickle.dump(podcast_data, f1)

podcast = pd.DataFrame(podcast_data.iloc[0,:]).T

opensmile_path = podcast['opensmile_path']
podcast_path = podcast['podcast_path']


doc_dict, podcast_sents_list, sents_dict = get_dicts(podcast)

updated_sents_dict = get_label(sents_dict, doc_dict)

podcast_data['SpeakerTag'].isnull().values.any()

In [ ]:
import pickle
pickle_off = open ("/content/drive/MyDrive/data/podcast_data.txt", "rb")
podcast_data = pickle.load(pickle_off)
len(podcast_data)

1250

In [ ]:

def get_dicts(df):   
  sents_dict = {}
  doc_dict = { i: {"Transcript": df.Transcript[i], "show_name": df.show_name[i], "Description": df.episode_description[i], "episode_uri": df.episode_uri[i], "podcast_path": df.podcast_path[i], "opensmile_path": df.opensmile_path[i] } for i in df.index }
  raw_docs = [ doc_dict[k]["Transcript"] for k in doc_dict.keys()]

  doc_sents = {}
  sents_list = []
  raw_sents = [] 
  i = 0
  min_sent_length = 15
  for k in tqdm(doc_dict.keys()):
    transcript = doc_dict[k]["Transcript"]  
    description = doc_dict[k]["Description"] 
    showname = doc_dict[k]["show_name"]
    episode_id = doc_dict[k]["episode_uri"]
    podcast_path = doc_dict[k]["podcast_path"]
    opensmile_path = doc_dict[k]["opensmile_path"]
    sents = retrieve_timestamped_transcript(podcast_path, opensmile_path)
   # sents = nlp(transcript).sents
    doc_sent_ids = [] 
    for index,sent in sents.iterrows():
        #print(len(sent[:]))
    #for sent in sents:
        if (len(sent['sentence'])) > min_sent_length:
      #if (len(sent)) > min_sent_length:
          sents_dict[i] = {"docid":k,"show_name":showname, "episode_id":episode_id, 'Description':description, 'Transcript': transcript, 'speakerTag': sent['speakerTag']}
          sents_dict[i].update(sent[:].to_dict())
        # sents_dict[0].update(sents.iloc[2,:].to_dict())
          #sents_dict[i] = {"docid":k, "text": str(sent)} 
          sents_list.append({"sentid":i,"show_name":showname,  "doc_id":k,"Description": description, "episode_id":episode_id,'text':sent['sentence'], 'speakerTag': sent['speakerTag'] })
        #list(sent.to_dict().items())
        
        i += 1  
         
  return doc_dict, sents_list, sents_dict



doc_dict, podcast_sents_list, sents_dict = get_dicts(podcast_data)

#sents_df = [] pd.DataFrame.from_dict(sents_dict).T.reset_index(drop=True)

print("# of sentences in dataset after labeling: " + str(len(podcast_sents_list)))
print("# of episodes indataset after labeling: " + str(len(pd.DataFrame.from_dict(sents_dict).T.groupby('episode_id').count())))
#print(pd.Series(sents_dict[0]))
print(len(podcast_sents_list))
print(len(sents_dict))



100%|██████████| 4197/4197 [13:40:06<00:00, 11.72s/it]


# of sentences in dataset after labeling: 1105560
# of episodes indataset after labeling: 4197
1105560
1105560


In [ ]:
podcast_data.groupby('show_name').count().head()

,episode_uri,Transcript,episode_description,opensmile_path,podcast_path
show_name,,,,,
#the100dayproject: explore your creativity,6,6,6,6,6
10+1 podcast,1,1,1,1,1
100% | a podcast for students,2,2,2,2,2
"33fuel podcast sports nutrition, training, motivation",3,3,3,3,3
4mics podcast,1,1,1,1,1


# **Data Preprocessing - Extracting Sentences with Spacy**
For each article, we extract sentences with Spacy

construct sentence dictionary and document dictionary
Note the min_sent_length hyper parameter - we only add sentences that have > 14 characters. This can be modified based on the expected sentence lengths at test time.

In [ ]:
os.makedirs("data/", exist_ok=True) 
os.makedirs("data/test", exist_ok=True) 
os.makedirs("data/train", exist_ok=True) 
os.makedirs("data/valid", exist_ok=True) 

#test_df.to_json("data/test/test.json")
#pd.DataFrame.from_dict(sents_dict).T.to_json("data/train/sents_dict.json")


In [ ]:
from rouge_score import rouge_scorer 
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

#df = df.dropna(axis = 0)

def get_rougue_score(text, highlights, metric="rougeL"):
  max_score = 0
  for h_text in highlights:
    score =  scorer.score(text, h_text)[metric].fmeasure
    # print(score, text, "\n \t" , h_text)
    if score > max_score:
      max_score = score 
  return max_score 

def get_label(sents_dict, doc_dict):

        for key,sent in tqdm(sents_dict.items()):
             sent_id, doc_id, episode_id, sentence = [],[],[],[]
             sent_id, doc_id, episode_id, sentence = key, sent["docid"], sent["episode_id"],sent["sentence"] 
             highlights = doc_dict[doc_id]["Description"].split("\n")
             doc = doc_dict[doc_id]["Transcript"]
             episode_id = doc_dict[doc_id]["episode_uri"]
             rouge_score = get_rougue_score(sentence, highlights) 
             #print(episode_id + " : " + str(rouge_score) + " " + str(sentence))
             sent.update({"rouge_score":rouge_score})
 
        return sents_dict
        

# sents_df = sents_df.groupby('episode_id').quantile(.90)  .filter(lambda x: (x.label_score.sum() > 5))


updated_sents_dict = get_label(sents_dict, doc_dict)



100%|██████████| 1105560/1105560 [50:40<00:00, 363.58it/s]


In [ ]:
sents_df = pd.DataFrame.from_dict(updated_sents_dict).T
#sents_df.iloc[:,7:-1] = sents_df.iloc[:,7:-1].apply(lambda x: (x - x.min()) / (x.max() - x.min()), axis = 0)
sents_df['label_range_perdoc'] = sents_df.groupby(['docid'])['rouge_score'].transform(lambda x: pd.qcut(x, q= [0, 0.875, 1], duplicates = 'drop'))
sents_df['label_score'] = sents_df.groupby(['docid'])['rouge_score'].transform(lambda x: pd.qcut(x, q= [0, 0.875, 1], labels = False, duplicates = 'drop'))
sents_df = sents_df.reset_index(drop = True)

sents_df = sents_df.reset_index().rename(columns={'index': 'sentence_id', 'docid':'doc_id'})
sents_df.groupby('episode_id')['label_score'].value_counts()
sents_df.groupby('episode_id')['label_range_perdoc'].value_counts()


In [ ]:
podcast = sents_df[sents_df['episode_id'] =='spotify:episode:3snXClDxVVaPU7VrgFBx5u']


from google.colab import data_table
data_table.enable_dataframe_formatter()

pd.options.display.max_rows = 999

podcast.iloc[]

,sentence_id,doc_id,show_name,episode_id,Description,Transcript,speakerTag,sentence,MEAN:F0semitoneFrom27.5Hz_sma3nz_amean,MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevNorm,...,MAX:loudnessPeaksPerSec,MAX:VoicedSegmentsPerSec,MAX:MeanVoicedSegmentLengthSec,MAX:StddevVoicedSegmentLengthSec,MAX:MeanUnvoicedSegmentLength,MAX:StddevUnvoicedSegmentLength,MAX:equivalentSoundLevel_dBp,rouge_score,label_range_perdoc,label_score
244061,244061,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,Outside my kashi's house in touch Ridge.,32.249957,0.05042,...,5.208333,6.25,0.193333,0.089938,0.106667,0.036551,-36.170082,0.04878,"(-0.001, 0.087]",0
244062,244062,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,Why have you come here today?,33.250621,0.076045,...,5.208333,4.166667,0.17,0.091924,0.11,0.07,-36.159229,0,"(-0.001, 0.087]",0
244063,244063,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,I'm here on principle my father supported in N...,30.783772,0.148223,...,4.166667,5.208333,0.203333,0.162138,0.365,0.255,-34.04731,0,"(-0.001, 0.087]",0
244064,244064,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,"And as far as I'm aware, my great-grandfather ...",30.746431,0.066094,...,5.208333,5.208333,0.41,0.32,0.27,0.19,-40.073296,0.105263,"(0.087, 0.196]",1
244065,244065,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,We just want to get him out from hiding behind...,30.944589,0.052208,...,6.25,4.166667,0.76,0.189912,0.51,0.214631,-43.947739,0.095238,"(0.087, 0.196]",1
244066,244066,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,I just want my gosh the out the club and anyth...,29.980331,0.027889,...,6.25,5.208333,0.236667,0.158605,0.37,0.26,-45.843353,0.042553,"(-0.001, 0.087]",0
244067,244067,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,To to hasten not I'm going to be a part of and...,30.112636,0.078828,...,7.291667,4.166667,0.26,0.114698,0.73,0.1772,-42.044449,0.036364,"(-0.001, 0.087]",0
244068,244068,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,I have to say what would you say to people who...,29.411739,0.067083,...,8.333334,4.166667,0.216667,0.121861,0.345,0.215,-39.480141,0.038462,"(-0.001, 0.087]",0
244069,244069,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,I would disagree.,25.730246,0.313165,...,4.166667,6.25,0.1025,0.077902,0.1175,0.111786,-37.690762,0,"(-0.001, 0.087]",0
244070,244070,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,There's nothing extreme in this.,29.264214,0.107214,...,8.333334,6.25,0.104,0.123709,0.07,0.039699,-41.561005,0.051282,"(-0.001, 0.087]",0


In [ ]:
#Split the dataset to train, val and test

episode_list = list(sents_df['doc_id'].unique())

train_ids = episode_list[0:int(len(episode_list)*0.8)]

valid_ids = episode_list[int(len(episode_list)*0.8):int(len(episode_list)*0.9)]

test_ids = episode_list[int(len(episode_list)*0.9):int(len(episode_list))]


train_df = sents_df.groupby('doc_id').filter(lambda x: x['doc_id'].values[0] in train_ids)
val_df = sents_df.groupby('doc_id').filter(lambda x: x['doc_id'].values[0] in valid_ids)
test_df = sents_df.groupby('doc_id').filter(lambda x: x['doc_id'].values[0] in test_ids)


print(train_df['doc_id'].nunique())
print(val_df['doc_id'].nunique())
print(test_df['doc_id'].nunique())

print('train-val-test')
print(len(train_df), len(val_df), len(test_df))

train_df

918
115
115
train-val-test
215431 23962 33877


,sentence_id,doc_id,show_name,episode_id,Description,Transcript,speakerTag,sentence,MEAN:F0semitoneFrom27.5Hz_sma3nz_amean,MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevNorm,...,MAX:loudnessPeaksPerSec,MAX:VoicedSegmentsPerSec,MAX:MeanVoicedSegmentLengthSec,MAX:StddevVoicedSegmentLengthSec,MAX:MeanUnvoicedSegmentLength,MAX:StddevUnvoicedSegmentLength,MAX:equivalentSoundLevel_dBp,rouge_score,label_range_perdoc,label_score
0,0,0,caspers place,spotify:episode:0zGAhzZLxAenuc5x1WYe8l,talk about my leaving of role play from the pa...,"Yeah, it's been a while since we've had a pod...",3,"Yeah, it's been a while since we've had a podc...",17.536679,0.099859,...,6.25,3.125,0.45,0.21276,0.75,0.085,-16.705429,0.07619,"(-0.001, 0.107]",0
1,1,0,caspers place,spotify:episode:0zGAhzZLxAenuc5x1WYe8l,talk about my leaving of role play from the pa...,"Yeah, it's been a while since we've had a pod...",3,It's time to bring this bag Paradise.,17.982735,0.276071,...,6.25,7.291667,0.1875,0.189654,0.15,0.148549,-19.564684,0.059406,"(-0.001, 0.107]",0
2,2,0,caspers place,spotify:episode:0zGAhzZLxAenuc5x1WYe8l,talk about my leaving of role play from the pa...,"Yeah, it's been a while since we've had a pod...",3,What's up guys is Casper and we are recording ...,19.057097,0.254715,...,8.333334,5.208333,0.355,0.167796,0.22,0.132246,-16.716869,0.056075,"(-0.001, 0.107]",0
3,3,0,caspers place,spotify:episode:0zGAhzZLxAenuc5x1WYe8l,talk about my leaving of role play from the pa...,"Yeah, it's been a while since we've had a pod...",3,It's been a while since I've done a podcast be...,18.854741,0.199866,...,7.291667,5.208333,0.45,0.245,0.18,0.103709,-17.609936,0.075472,"(-0.001, 0.107]",0
4,4,0,caspers place,spotify:episode:0zGAhzZLxAenuc5x1WYe8l,talk about my leaving of role play from the pa...,"Yeah, it's been a while since we've had a pod...",3,We've been doing a lot of changes since I've l...,18.099544,0.307787,...,5.208333,5.208333,0.95,0.38,0.08,0.03,-16.896196,0.037736,"(-0.001, 0.107]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215426,215426,917,the all talk podcast,spotify:episode:6inawgjruAYSb4qXwm3oqY,"on this week's episode, jeff pays tribute to t...",Welcome in to the aortic body cast. This is y...,2,Enjoy your Friday.,27.941633,0.149154,...,3.125,3.125,0.273333,0.245,0.4,0.055578,-33.360039,0,"(-0.001, 0.119]",0
215427,215427,917,the all talk podcast,spotify:episode:6inawgjruAYSb4qXwm3oqY,"on this week's episode, jeff pays tribute to t...",Welcome in to the aortic body cast. This is y...,2,Enjoy your weekend.,30.295418,0.312897,...,4.166667,2.083333,0.4,0.26,0.3,0.28,-32.955475,0.031746,"(-0.001, 0.119]",0
215428,215428,917,the all talk podcast,spotify:episode:6inawgjruAYSb4qXwm3oqY,"on this week's episode, jeff pays tribute to t...",Welcome in to the aortic body cast. This is y...,2,"You've had a rough week, but we're going to tu...",30.750542,0.157191,...,8.333334,3.125,0.67,0.265,0.71,0.193276,-35.314411,0.076923,"(-0.001, 0.119]",0
215429,215429,917,the all talk podcast,spotify:episode:6inawgjruAYSb4qXwm3oqY,"on this week's episode, jeff pays tribute to t...",Welcome in to the aortic body cast. This is y...,2,Let's go Chiefs baby.,31.453845,0.139175,...,2.083333,2.083333,0.32,0.16,0.305,0.175,-26.599789,0.061538,"(-0.001, 0.119]",0


In [ ]:
sents_df[sents_df['label_score']== 1].groupby(['episode_id']).size().sort_values(ascending = False)[:1000]

'spotify:episode:0XwEklMtGIDXmEnlKZJ3Qc'
print(sents_df[sents_df['episode_id'] =='spotify:episode:3snXClDxVVaPU7VrgFBx5u'].loc[70870,'Transcript'])
sents_df.groupby(['episode_id', 'label_score']).size().sort_values( ascending = False)[:100]

,sentence_id,doc_id,show_name,episode_id,Description,Transcript,speakerTag,sentence,MEAN:F0semitoneFrom27.5Hz_sma3nz_amean,MEAN:F0semitoneFrom27.5Hz_sma3nz_stddevNorm,...,MAX:loudnessPeaksPerSec,MAX:VoicedSegmentsPerSec,MAX:MeanVoicedSegmentLengthSec,MAX:StddevVoicedSegmentLengthSec,MAX:MeanUnvoicedSegmentLength,MAX:StddevUnvoicedSegmentLength,MAX:equivalentSoundLevel_dBp,rouge_score,label_range_perdoc,label_score
244061,244061,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,Outside my kashi's house in touch Ridge.,32.249957,0.05042,...,5.208333,6.25,0.193333,0.089938,0.106667,0.036551,-36.170082,0.04878,"(-0.001, 0.087]",0
244062,244062,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,Why have you come here today?,33.250621,0.076045,...,5.208333,4.166667,0.17,0.091924,0.11,0.07,-36.159229,0,"(-0.001, 0.087]",0
244063,244063,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,I'm here on principle my father supported in N...,30.783772,0.148223,...,4.166667,5.208333,0.203333,0.162138,0.365,0.255,-34.04731,0,"(-0.001, 0.087]",0
244064,244064,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,"And as far as I'm aware, my great-grandfather ...",30.746431,0.066094,...,5.208333,5.208333,0.41,0.32,0.27,0.19,-40.073296,0.105263,"(0.087, 0.196]",1
244065,244065,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,We just want to get him out from hiding behind...,30.944589,0.052208,...,6.25,4.166667,0.76,0.189912,0.51,0.214631,-43.947739,0.095238,"(0.087, 0.196]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244142,244142,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,So I've got no idea what we want to get in bef...,32.481143,0.113852,...,7.291667,5.208333,0.44,0.355,0.233333,0.231565,-39.790634,0.051282,"(-0.001, 0.087]",0
244143,244143,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,"I don't want them, but he wants to help it.",33.21881,0.120182,...,7.291667,4.166667,0.42,0.37,0.17,0.05,-40.394642,0.045455,"(-0.001, 0.087]",0
244144,244144,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,I don't want them but you know needs must need...,31.934112,0.188654,...,9.375,6.25,0.236667,0.23041,0.216667,0.232702,-40.875439,0.040816,"(-0.001, 0.087]",0
244145,244145,1050,nufc outcast - united voice for newcastle,spotify:episode:3snXClDxVVaPU7VrgFBx5u,london magpie group protest - live-ish from to...,Outside my kashi's house in touch Ridge. Why ...,3,"Chile's chose us your time, excellent stuff an...",29.215992,0.130313,...,5.208333,6.25,0.77,0.2,0.17,0.095,-20.074259,0.044444,"(-0.001, 0.087]",0


In [ ]:
#RE-preprocess for balancing the data

import pickle
print( "Train, test shape before repreprocess", train_df.shape, val_df.shape)
print("Total number of episodes before re-preprocess :" + str(len(train_df.groupby('episode_id')) + len(val_df.groupby('episode_id'))))

#Filter episodes which has the most number of sentences which have higher than 90th percentiles of the podcast episode

def filterepisodes(df, threshold):
    pos_df, neg_df = [], []
    ids = []
    df = df.dropna(axis = 0)
    ids = df[df['label_score']== 1].groupby(['episode_id']).size().sort_values(ascending = False)[:threshold].index.tolist()  
    df = df.groupby('episode_id').filter(lambda x: x['episode_id'].values[0] in ids)
    pos_df = df[df.label_score == 1]
    neg_df = df[df.label_score == 0].groupby('episode_id').sample(frac = .25)
    df = pos_df.append(neg_df)

    return pos_df, neg_df, df

pos_df, neg_df, train_df = filterepisodes(train_df,threshold = 800)

print("Negative sample size for training:", len(neg_df))
print("Positive sample size for training:", len(pos_df))
print("Balanced sample size for training:", len(train_df))

print("Total number of episodes in train set:" + str(len(train_df.groupby('episode_id'))))

### val prepare

val_pos_df, val_neg_df, val_df = filterepisodes(val_df,threshold = 100)

print("Negative sample size for test:", len(val_neg_df))
print("Positive sample size for test:", len(val_pos_df))
print("Balanced sample size for test:", len(val_df))

print("Total number of episodes in test set:" + str(len(val_df.groupby('episode_id'))))


### test prepare

test_pos_df, test_neg_df, test_df = filterepisodes(test_df,threshold = 100)

print("Negative sample size for test:", len(test_neg_df))
print("Positive sample size for test:", len(test_pos_df))
print("Balanced sample size for test:", len(test_df))

print("Total number of episodes in test set:" + str(len(test_df.groupby('episode_id'))))






Train, test shape before repreprocess (215431, 275) (23962, 275)
Total number of episodes before re-preprocess :1033
Negative sample size for training: 46997
Positive sample size for training: 21039
Balanced sample size for training: 68036
Total number of episodes in train set:800
Negative sample size for test: 5129
Positive sample size for test: 2299
Balanced sample size for test: 7428
Total number of episodes in test set:100
Negative sample size for test: 7259
Positive sample size for test: 3231
Balanced sample size for test: 10490
Total number of episodes in test set:100


In [ ]:
import pickle

with open('train_800podcastepisodes_SpeakerTag.txt', 'wb') as f1:
   pickle.dump(train_df, f1)

with open('test_100podcastepisodes_SpeakerTag.txt', 'wb') as f1:
   pickle.dump(test_df, f1)

with open('val_100podcastepisodes_SpeakerTag.txt', 'wb') as f1:
   pickle.dump(val_df, f1)



In [ ]:
#train_df.groupby(['episode_id', 'label_score']).size().sort_values(ascending = True)

'''


val_df.groupby(['episode_id']).sum().sort_values(by='label_score')

with open('train_1000podcasts.txt', 'wb') as f1:
   pickle.dump(train_df, f1)


### test prepare

pos_df, neg_df, test_df = filterepisodes(val_df,threshold = 200)

print("Negative sample size for test:", len(neg_df))
print("Positive sample size for test:", len(pos_df))
print("Balanced sample size for test:", len(test_df))

print("Total number of episodes in test set:" + str(len(test_df.groupby('episode_id'))))

test_df.groupby(['episode_id']).sum().sort_values(by='label_score')

with open('train_1000podcasts.txt', 'wb') as f1:
   pickle.dump(train_df, f1)

with open('test_200podcasts.txt', 'wb') as f1:
   pickle.dump(test_df, f1)   

#from sklearn.model_selection import train_test_split

#train_df, test_df = train_test_split(total_set, test_size=0.2)

'''


#df[df['label_score']== 1].groupby(['episode_id']).size().sort_values(ascending = False)[:threshold][::10]